# Combinação de modelos - Parte 2 (GBM - Gradient boosting machines)

1. Cite 5 diferenças entre o AdaBoost e o GBM (Gradient boosting machines).

- O GBM inicia com a média da variável resposta/target, diferentemente do AdaBoost que inicia com a geração de Stumps;
  
- A técnica GBM utiliza florestas de árvore que podem possuir variados valores de profundidades e número mínimo de amostras por folha. O AdaBoost utiliza uma floresta de stumps, ou seja, árvores de profundidade 1 e número mínimo de amostras por folha igual a 2;

- Todas as respostas possuem um multiplicador em comum chamado *learning-rate* (*eta*) e não pesos diferentes para cada observação (como é feito do Adaboost);

- O modelo GBM faz uso de resíduos. Diferentemente do AdaBoost, que ajusta os pesos das instâncias a cada observação, o GBM busca ajustar o novo preditor aos erros residuais cometidos pelo preditor anterior;

- Um outro ponto que destaca a diferença entre as duas técnicas é a forma como ambas abordam as saídas dos modelos. Para o AdaBoost cada resposta tem um peso diferente, para o GBM as respostas tem um multiplicador comum e o adicional de que cada resposta subsequente é ajustada com base na resposta anterior para o cálculo do resíduo. Vamos considerar o cálculo do quarto resíduo na sequência do modelo (caso hipotético), assim teremos: $resíduo_4 = y - [predito_1 + eta*predito_2 + eta*predito_3]$, sendo que o valor predito_1 é a média da variável resposta (isso para análises de regressão).

2. Acesse o [Scikit-learn – GBM](https://scikit-learn.org/stable/modules/ensemble.html), leia a explicação (traduza se for preciso) e crie um jupyter notebook contendo o exemplo de classificação e de regressão do GBM.

In [ ]:
# Bibliotecas
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [12]:
%%time

# Testando o GBM classifier

X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

CPU times: total: 297 ms
Wall time: 301 ms


0.913

3. (Opcional) Utilize o GridSearch para encontrar os melhores hyperparametros para o conjunto de dados do exemplo

In [14]:
%%time

X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

gb = GradientBoostingClassifier()

parametros = {
    'n_estimators': list(range(1, 1001, 100)),
    'min_samples_leaf': list(range(1, 50, 5)),
    'learning_rate': [0.04, 0.06, .1]
}

grid = GridSearchCV(estimator=gb,
                    param_grid=parametros,
                    scoring='roc_auc',
                    verbose=False,
                    cv=2)

grid.fit(X_train, y_train.ravel())

best_params = grid.best_params_
print(f'Melhores parâmetros: {best_params}')


score = grid.best_estimator_.score(X_test, y_test)
score

Melhores parâmetros: {'learning_rate': 0.06, 'min_samples_leaf': 16, 'n_estimators': 901}
CPU times: total: 17min 10s
Wall time: 17min 17s


0.9155

3. Cite 5 hiperparâmetros importantes no GBM

Os hiperparâmetros do método são semelhantes aos do AdaBoost.
- **loss**: *{‘squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’}, default=’squared_error’* - É a função de perda a ser otimizada;
- **learning_rate**: Reduz a contribuição de cada árvore;
- **n_estimator**: Representa o número de estágios de boosting a serem realizados. O GBM é robusto ao overfitting e um maior o número de estimadores/árvores geralmente resulta em melhor desempenho;
- **subsample**: A fração das amostras a serem utilizadas para ajustar os *weak learners* ou *base learners*;
- **criterion**: *{‘friedman_mse’, ‘squared_error’}, default=’friedman_mse’* - Função utilizada para medir a qualidade de uma divisão, um *split*.

5. Acessando o artigo do Jerome Friedman ([Stochastic](https://jerryfriedman.su.domains/ftp/stobst.pdf)) e pensando no nome dado ao Stochastic GBM, qual é a maior diferença entre os dois algoritmos?

- Basicamente a diferença entre os dois algoritmos se encontra no treinamento dos *weak lerners*. O GBM utiliza todos os dados de treino nessa tarefa, já o *stochastic gradient boosting* (SGBM) utiliza uma subamostragem dos dados de treino para o desenvolvimento dos *weak lerners*. Esse último é capaz de aprimorar o retorno em função da adição de aleatoriedade e variância, pois as subamostras tendem a ser menos correlacionadas, o que também ajuda a evitar o overfitting no algoritmo. 